In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import geopandas as gpd

FIRST READ DATA

In [2]:
df_counties_original = pd.read_csv("./us-counties.csv", dtype={"date": np.string_, 
                                                             "county": np.string_,
                                                             "state": np.string_,
                                                             "fips": np.string_,
                                                            "cases": np.int32,
                                                            "deaths": np.int32})
#Tansform Unknown fips to '00000' 
df_counties_original['fips'] = df_counties_original['fips'].replace({np.nan: '00000'})
df_counties_original.sort_values("fips", ascending=True)


,date,county,state,fips,cases,deaths
11424,2020-03-25,Unknown,Louisiana,00000,12,1
31333,2020-04-04,Unknown,Guam,00000,223,4
28575,2020-04-03,Unknown,Arizona,00000,0,4
4334,2020-03-18,Unknown,Virgin Islands,00000,3,0
16044,2020-03-28,Unknown,Colorado,00000,167,0
...,...,...,...,...,...,...
28488,2020-04-02,Washakie,Wyoming,56043,2,0
30845,2020-04-03,Washakie,Wyoming,56043,2,0
33255,2020-04-04,Washakie,Wyoming,56043,2,0
15835,2020-03-27,Washakie,Wyoming,56043,1,0


Generate GeoJSON

In [3]:
pivot_counties = pd.pivot_table(df_counties_original , index=['state','county','fips'],
                       columns=['date'])
pivot_counties = pivot_counties.replace(np.nan, 0)
pivot_counties


cases                                   \
date                     2020-01-21 2020-01-22 2020-01-23 2020-01-24   
state   county     fips                                                
Alabama Autauga    01001        0.0        0.0        0.0        0.0   
        Baldwin    01003        0.0        0.0        0.0        0.0   
        Barbour    01005        0.0        0.0        0.0        0.0   
        Bibb       01007        0.0        0.0        0.0        0.0   
        Blount     01009        0.0        0.0        0.0        0.0   
...                             ...        ...        ...        ...   
Wyoming Sublette   56035        0.0        0.0        0.0        0.0   
        Sweetwater 56037        0.0        0.0        0.0        0.0   
        Teton      56039        0.0        0.0        0.0        0.0   
        Uinta      56041        0.0        0.0        0.0        0.0   
        Washakie   56043        0.0        0.0        0.0        0.0   

                                                                      \
date                     2020-01-25 2020-01-26 2020-01-27 2020-01-28   
state   county     fips                                                
Alabama Autauga    01001        0.0        0.0        0.0        0.0   
        Baldwin    01003        0.0        0.0        0.0        0.0   
        Barbour    01005        0.0        0.0        0.0        0.0   
        Bibb       01007        0.0        0.0        0.0        0.0   
        Blount     01009        0.0        0.0        0.0        0.0   
...                             ...        ...        ...        ...   
Wyoming Sublette   56035        0.0        0.0        0.0        0.0   
        Sweetwater 56037        0.0        0.0        0.0        0.0   
        Teton      56039        0.0        0.0        0.0        0.0   
        Uinta      56041        0.0        0.0        0.0        0.0   
        Washakie   56043        0.0        0.0        0.0        0.0   

                                                ...     deaths             \
date                     2020-01-29 2020-01-30  ... 2020-03-28 2020-03-29   
state   county     fips                         ...                         
Alabama Autauga    01001        0.0        0.0  ...        0.0        0.0   
        Baldwin    01003        0.0        0.0  ...        0.0        0.0   
        Barbour    01005        0.0        0.0  ...        0.0        0.0   
        Bibb       01007        0.0        0.0  ...        0.0        0.0   
        Blount     01009        0.0        0.0  ...        0.0        0.0   
...                             ...        ...  ...        ...        ...   
Wyoming Sublette   56035        0.0        0.0  ...        0.0        0.0   
        Sweetwater 56037        0.0        0.0  ...        0.0        0.0   
        Teton      56039        0.0        0.0  ...        0.0        0.0   
        Uinta      56041        0.0        0.0  ...        0.0        0.0   
        Washakie   56043        0.0        0.0  ...        0.0        0.0   

                                                                      \
date                     2020-03-30 2020-03-31 2020-04-01 2020-04-02   
state   county     fips                                                
Alabama Autauga    01001        0.0        0.0        0.0        0.0   
        Baldwin    01003        0.0        0.0        1.0        1.0   
        Barbour    01005        0.0        0.0        0.0        0.0   
        Bibb       01007        0.0        0.0        0.0        0.0   
        Blount     01009        0.0        0.0        0.0        0.0   
...                             ...        ...        ...        ...   
Wyoming Sublette   56035        0.0        0.0        0.0        0.0   
        Sweetwater 56037        0.0        0.0        0.0        0.0   
        Teton      56039        0.0        0.0        0.0        0.0   
        Uinta      56041        0.0        0.0        0.0        0.0 

Get Date Info

In [4]:
date= pivot_counties['cases'].columns
date

Index(['2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25',
       '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30',
       '2020-01-31', '2020-02-01', '2020-02-02', '2020-02-03', '2020-02-04',
       '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-08', '2020-02-09',
       '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14',
       '2020-02-15', '2020-02-16', '2020-02-17', '2020-02-18', '2020-02-19',
       '2020-02-20', '2020-02-21', '2020-02-22', '2020-02-23', '2020-02-24',
       '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-28', '2020-02-29',
       '2020-03-01', '2020-03-02', '2020-03-03', '2020-03-04', '2020-03-05',
       '2020-03-06', '2020-03-07', '2020-03-08', '2020-03-09', '2020-03-10',
       '2020-03-11', '2020-03-12', '2020-03-13', '2020-03-14', '2020-03-15',
       '2020-03-16', '2020-03-17', '2020-03-18', '2020-03-19', '2020-03-20',
       '2020-03-21', '2020-03-22', '2020-03-23', '2020-03-24', '2020-03-25',

In [5]:
date_str = np.sort(date)
dt_str_start=np.min(date_str)
dt_str_end=np.max(date_str)

In [6]:
dt_start = datetime.strptime(dt_str_start, "%Y-%m-%d")
dt_end = datetime.strptime(dt_str_end, "%Y-%m-%d")
dt_range = pd.date_range(start=dt_start,end=dt_end)
print(len(dt_range), dt_range)
dt_range_str = list(map(lambda x: x.strftime("%Y-%m-%d"), dt_range.tolist()))
print(len(dt_range_str), dt_range_str)

77 DatetimeIndex(['2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24',
               '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28',
               '2020-01-29', '2020-01-30', '2020-01-31', '2020-02-01',
               '2020-02-02', '2020-02-03', '2020-02-04', '2020-02-05',
               '2020-02-06', '2020-02-07', '2020-02-08', '2020-02-09',
               '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13',
               '2020-02-14', '2020-02-15', '2020-02-16', '2020-02-17',
               '2020-02-18', '2020-02-19', '2020-02-20', '2020-02-21',
               '2020-02-22', '2020-02-23', '2020-02-24', '2020-02-25',
               '2020-02-26', '2020-02-27', '2020-02-28', '2020-02-29',
               '2020-03-01', '2020-03-02', '2020-03-03', '2020-03-04',
               '2020-03-05', '2020-03-06', '2020-03-07', '2020-03-08',
               '2020-03-09', '2020-03-10', '2020-03-11', '2020-03-12',
               '2020-03-13', '2020-03-14', '2020-03-15', '2020-03-16',
   

Add Cases Time Series, First Case Date, Death Time Series, First Death Date

In [7]:
# import json
# pivot_counties['cases_ts'] = json.dumps({"values": pivot_counties['cases'].values.tolist()[0]})
# pivot_counties['deaths_ts'] =  json.dumps({"values": pivot_counties['deaths'].values.tolist()[0]})

pivot_counties['cases_ts'] =  pivot_counties['cases'].values.tolist()
pivot_counties['cases_ts'] = pivot_counties['cases_ts'].apply(lambda x: ','.join(map(str, x)))
pivot_counties['deaths_ts'] =  pivot_counties['deaths'].values.tolist()
pivot_counties['deaths_ts'] = pivot_counties['deaths_ts'].apply(lambda x: ','.join(map(str, x)))

pivot_counties

cases                                   \
date                     2020-01-21 2020-01-22 2020-01-23 2020-01-24   
state   county     fips                                                
Alabama Autauga    01001        0.0        0.0        0.0        0.0   
        Baldwin    01003        0.0        0.0        0.0        0.0   
        Barbour    01005        0.0        0.0        0.0        0.0   
        Bibb       01007        0.0        0.0        0.0        0.0   
        Blount     01009        0.0        0.0        0.0        0.0   
...                             ...        ...        ...        ...   
Wyoming Sublette   56035        0.0        0.0        0.0        0.0   
        Sweetwater 56037        0.0        0.0        0.0        0.0   
        Teton      56039        0.0        0.0        0.0        0.0   
        Uinta      56041        0.0        0.0        0.0        0.0   
        Washakie   56043        0.0        0.0        0.0        0.0   

                                                                      \
date                     2020-01-25 2020-01-26 2020-01-27 2020-01-28   
state   county     fips                                                
Alabama Autauga    01001        0.0        0.0        0.0        0.0   
        Baldwin    01003        0.0        0.0        0.0        0.0   
        Barbour    01005        0.0        0.0        0.0        0.0   
        Bibb       01007        0.0        0.0        0.0        0.0   
        Blount     01009        0.0        0.0        0.0        0.0   
...                             ...        ...        ...        ...   
Wyoming Sublette   56035        0.0        0.0        0.0        0.0   
        Sweetwater 56037        0.0        0.0        0.0        0.0   
        Teton      56039        0.0        0.0        0.0        0.0   
        Uinta      56041        0.0        0.0        0.0        0.0   
        Washakie   56043        0.0        0.0        0.0        0.0   

                                                ...     deaths             \
date                     2020-01-29 2020-01-30  ... 2020-03-30 2020-03-31   
state   county     fips                         ...                         
Alabama Autauga    01001        0.0        0.0  ...        0.0        0.0   
        Baldwin    01003        0.0        0.0  ...        0.0        0.0   
        Barbour    01005        0.0        0.0  ...        0.0        0.0   
        Bibb       01007        0.0        0.0  ...        0.0        0.0   
        Blount     01009        0.0        0.0  ...        0.0        0.0   
...                             ...        ...  ...        ...        ...   
Wyoming Sublette   56035        0.0        0.0  ...        0.0        0.0   
        Sweetwater 56037        0.0        0.0  ...        0.0        0.0   
        Teton      56039        0.0        0.0  ...        0.0        0.0   
        Uinta      56041        0.0        0.0  ...        0.0        0.0   
        Washakie   56043        0.0        0.0  ...        0.0        0.0   

                                                                      \
date                     2020-04-01 2020-04-02 2020-04-03 2020-04-04   
state   county     fips                                                
Alabama Autauga    01001        0.0        0.0        0.0        0.0   
        Baldwin    01003        1.0        1.0        1.0        1.0   
        Barbour    01005        0.0        0.0        0.0        0.0   
        Bibb       01007        0.0        0.0        0.0        0.0   
        Blount     01009        0.0        0.0        0.0        0.0   
...                             ...        ...        ...        ...   
Wyoming Sublette   56035        0.0        0.0        0.0        0.0   
        Sweetwater 56037        0.0        0.0        0.0        0.0   
        Teton      56039        0.0        0.0        0.0        0.0   
        Uinta      56041        0.0        0.0        0.0        0.0 

In [8]:
yesterday = date.values[-2]
pivot_counties['today_case'] = pivot_counties['cases'][dt_str_end]
pivot_counties['today_new_case'] = pivot_counties['cases'][dt_str_end] - pivot_counties['cases'][yesterday]
pivot_counties['today_death'] = pivot_counties['deaths'][dt_str_end]
pivot_counties['today_new_death'] = pivot_counties['deaths'][dt_str_end] - pivot_counties['deaths'][yesterday]

In [9]:
pivot_counties['dt_first_case'] = (pivot_counties['cases'] > 0).idxmax(axis=1)
pivot_counties['dt_first_death'] = (pivot_counties['deaths'] > 0).idxmax(axis=1)
#For death reports, deal with no deaths counties
pivot_counties.loc[pivot_counties['deaths'].iloc[:, -1] <= 0, 'dt_first_death'] = np.nan
pivot_counties.head(5)

cases                                              \
date                  2020-01-21 2020-01-22 2020-01-23 2020-01-24 2020-01-25   
state   county  fips                                                           
Alabama Autauga 01001        0.0        0.0        0.0        0.0        0.0   
        Baldwin 01003        0.0        0.0        0.0        0.0        0.0   
        Barbour 01005        0.0        0.0        0.0        0.0        0.0   
        Bibb    01007        0.0        0.0        0.0        0.0        0.0   
        Blount  01009        0.0        0.0        0.0        0.0        0.0   

                                                                              \
date                  2020-01-26 2020-01-27 2020-01-28 2020-01-29 2020-01-30   
state   county  fips                                                           
Alabama Autauga 01001        0.0        0.0        0.0        0.0        0.0   
        Baldwin 01003        0.0        0.0        0.0        0.0        0.0   
        Barbour 01005        0.0        0.0        0.0        0.0        0.0   
        Bibb    01007        0.0        0.0        0.0        0.0        0.0   
        Blount  01009        0.0        0.0        0.0        0.0        0.0   

                       ...     deaths             \
date                   ... 2020-04-05 2020-04-06   
state   county  fips   ...                         
Alabama Autauga 01001  ...        0.0        1.0   
        Baldwin 01003  ...        1.0        1.0   
        Barbour 01005  ...        0.0        0.0   
        Bibb    01007  ...        0.0        0.0   
        Blount  01009  ...        0.0        0.0   

                                                                cases_ts  \
date                                                                       
state   county  fips                                                       
Alabama Autauga 01001  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....   
        Baldwin 01003  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....   
        Barbour 01005  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....   
        Bibb    01007  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....   
        Blount  01009  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....   

                                                               deaths_ts  \
date                                                                       
state   county  fips                                                       
Alabama Autauga 01001  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....   
        Baldwin 01003  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....   
        Barbour 01005  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....   
        Bibb    01007  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....   
        Blount  01009  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....   

                      today_case today_new_case today_death today_new_death  \
date                                                                          
state   county  fips                                                          
Alabama Autauga 01001       12.0            0.0         1.0             1.0   
        Baldwin 01003       38.0            4.0         1.0             0.0   
        Barbour 01005        3.0            1.0         0.0             0.0   
        Bibb    01007        7.0            0.0         0.0             0.0   
        Blount  01009       10.0            0.0         0.0             0.0   

                      dt_first_case dt_first_death  
date                                                
state   county  fips                                
Alabama Autauga 01001    2020-03-24     2020-04-06  
        Baldwin 01003    2020-03-14     2020-04-01  
        Barbour 01005    2020-04-03            NaN  
        Bibb    01007    2020-03-30            NaN  
        Blount  01009    2020-03-25            NaN  

[5 rows x 162 columns]

Read County Boundary GeoJSON file

In [10]:
old_counties_geojson_df = gpd.read_file(r"./counties_update_new.geojson")
old_counties_geojson_df.head(5)

,NAME,state_name,STATEFP,COUNTYNS,population,geometry
0,Bladen,North Carolina,37,01026336,33778,"MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ..."
1,Stanly,North Carolina,37,01025844,61114,"MULTIPOLYGON (((-80.49738 35.20210, -80.29542 ..."
2,Summit,Ohio,39,01074088,541810,"MULTIPOLYGON (((-81.68699 41.13596, -81.68495 ..."
3,Sullivan,Pennsylvania,42,01213687,6177,"MULTIPOLYGON (((-76.81373 41.59003, -76.22014 ..."
4,Upshur,Texas,48,01384015,40769,"MULTIPOLYGON (((-95.15274 32.66095, -95.15211 ..."


Old Data Structure

In [11]:
report_df = pivot_counties[['cases_ts','deaths_ts','dt_first_case','dt_first_death','today_case','today_new_case', 'today_death','today_new_death']]
report_df = report_df.reset_index()
report_df.columns = ['state','county','fips','cases_ts','deaths_ts','dt_first_case','dt_first_death','today_case','today_new_case', 'today_death','today_new_death']
report_df.head(5)

,state,county,fips,cases_ts,deaths_ts,dt_first_case,dt_first_death,today_case,today_new_case,today_death,today_new_death
0,Alabama,Autauga,01001,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-24,2020-04-06,12.0,0.0,1.0,1.0
1,Alabama,Baldwin,01003,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-14,2020-04-01,38.0,4.0,1.0,0.0
2,Alabama,Barbour,01005,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-04-03,NaN,3.0,1.0,0.0,0.0
3,Alabama,Bibb,01007,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-30,NaN,7.0,0.0,0.0,0.0
4,Alabama,Blount,01009,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-25,NaN,10.0,0.0,0.0,0.0


In [12]:
final_df = pd.merge(old_counties_geojson_df[["NAME", "state_name", "population", "geometry"]], report_df, how='left', left_on=['NAME','state_name'], right_on = ['county','state'])
final_df.columns

Index(['NAME', 'state_name', 'population', 'geometry', 'state', 'county',
       'fips', 'cases_ts', 'deaths_ts', 'dt_first_case', 'dt_first_death',
       'today_case', 'today_new_case', 'today_death', 'today_new_death'],
      dtype='object')

In [13]:
final_df=final_df[["NAME", "state_name", "population", "fips", "dt_first_case", "dt_first_death", "cases_ts", "deaths_ts", 'today_case','today_new_case', 'today_death','today_new_death', "geometry"]]
final_df['dt_start'] = dt_str_start
final_df['dt_end'] = dt_str_end
final_df['dt_unit'] = "day"
final_df

,NAME,state_name,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,geometry,dt_start,dt_end,dt_unit
0,Bladen,North Carolina,33778,37017,2020-03-29,NaN,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",1.0,0.0,0.0,0.0,"MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ...",2020-01-21,2020-04-06,day
1,Stanly,North Carolina,61114,37167,2020-03-20,NaN,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",8.0,1.0,0.0,0.0,"MULTIPOLYGON (((-80.49738 35.20210, -80.29542 ...",2020-01-21,2020-04-06,day
2,Summit,Ohio,541810,39153,2020-03-14,2020-03-27,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",189.0,5.0,10.0,1.0,"MULTIPOLYGON (((-81.68699 41.13596, -81.68495 ...",2020-01-21,2020-04-06,day
3,Sullivan,Pennsylvania,6177,42113,2020-04-04,NaN,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",1.0,0.0,0.0,0.0,"MULTIPOLYGON (((-76.81373 41.59003, -76.22014 ...",2020-01-21,2020-04-06,day
4,Upshur,Texas,40769,48459,2020-03-20,NaN,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",3.0,0.0,0.0,0.0,"MULTIPOLYGON (((-95.15274 32.66095, -95.15211 ...",2020-01-21,2020-04-06,day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3211,Lajas,Puerto Rico,23315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-67.10904 18.05608, -67.06791 ...",2020-01-21,2020-04-06,day
3212,Seward,Nebraska,17127,31159,2020-04-05,NaN,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",1.0,0.0,0.0,0.0,"MULTIPOLYGON (((-97.36812 41.04695, -96.91094 ...",2020-01-21,2020-04-06,day
3213,Burke,North Carolina,89712,37023,2020-03-25,2020-04-05,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",25.0,10.0,1.0,0.0,"MULTIPOLYGON (((-81.90665 35.88338, -81.94319 ...",2020-01-21,2020-04-06,day
3214,Sumter,Georgia,30352,13261,2020-03-20,2020-03-28,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",90.0,7.0,4.0,0.0,"MULTIPOLYGON (((-84.43301 32.04196, -84.43121 ...",2020-01-21,2020-04-06,day


In [14]:
final_df[pd.notnull(final_df["cases_ts"])]['dt_start'] = dt_str_start
final_df

<ipython-input-14-c00da5d1f919>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df[pd.notnull(final_df["cases_ts"])]['dt_start'] = dt_str_start


,NAME,state_name,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,geometry,dt_start,dt_end,dt_unit
0,Bladen,North Carolina,33778,37017,2020-03-29,NaN,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",1.0,0.0,0.0,0.0,"MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ...",2020-01-21,2020-04-06,day
1,Stanly,North Carolina,61114,37167,2020-03-20,NaN,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",8.0,1.0,0.0,0.0,"MULTIPOLYGON (((-80.49738 35.20210, -80.29542 ...",2020-01-21,2020-04-06,day
2,Summit,Ohio,541810,39153,2020-03-14,2020-03-27,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",189.0,5.0,10.0,1.0,"MULTIPOLYGON (((-81.68699 41.13596, -81.68495 ...",2020-01-21,2020-04-06,day
3,Sullivan,Pennsylvania,6177,42113,2020-04-04,NaN,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",1.0,0.0,0.0,0.0,"MULTIPOLYGON (((-76.81373 41.59003, -76.22014 ...",2020-01-21,2020-04-06,day
4,Upshur,Texas,40769,48459,2020-03-20,NaN,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",3.0,0.0,0.0,0.0,"MULTIPOLYGON (((-95.15274 32.66095, -95.15211 ...",2020-01-21,2020-04-06,day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3211,Lajas,Puerto Rico,23315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-67.10904 18.05608, -67.06791 ...",2020-01-21,2020-04-06,day
3212,Seward,Nebraska,17127,31159,2020-04-05,NaN,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",1.0,0.0,0.0,0.0,"MULTIPOLYGON (((-97.36812 41.04695, -96.91094 ...",2020-01-21,2020-04-06,day
3213,Burke,North Carolina,89712,37023,2020-03-25,2020-04-05,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",25.0,10.0,1.0,0.0,"MULTIPOLYGON (((-81.90665 35.88338, -81.94319 ...",2020-01-21,2020-04-06,day
3214,Sumter,Georgia,30352,13261,2020-03-20,2020-03-28,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",90.0,7.0,4.0,0.0,"MULTIPOLYGON (((-84.43301 32.04196, -84.43121 ...",2020-01-21,2020-04-06,day


In [15]:
final_df.to_file(r"./nyt_counties_data.geojson", driver='GeoJSON', encoding='utf-8')
print("done")

done


New Data Structure (Full version)

In [16]:
# pivot_counties.columns = ['_'.join(col).strip() for col in pivot_counties.columns.values]
# report_df_2 = pivot_counties.reset_index()

In [17]:
# report_df_2.head(5)

In [18]:
# pivot_counties.head(5)

In [19]:
# final_df_2 = pd.merge(old_counties_geojson_df[["NAME", "state_name", "population", "geometry"]], report_df_2, how='right', left_on=['NAME','state_name'], right_on = ['county','state'])
# final_df_2 = final_df_2.drop(['state','county'], axis = 1)
# final_df_2['dt_start'] = dt_start
# final_df_2['dt_end'] = dt_end
# final_df_2.head(5)

In [20]:
# final_df_2.to_file(r"./nyt_counties_data_full.geojson", driver='GeoJSON', encoding='utf-8')
# print("done")

Generate JSON file

In [21]:
# county_list = []
# for state in df_counties_original.state.unique():
#     for county in df_counties_original.loc[(df_counties_original['state'] == state)].county.unique():
#         county_list.append({"name": county, "state": state})
 
# print(len(county_list), county_list)


In [22]:
# data_df = report_df.set_index(report_df["county"]+", "+report_df["state"])
# data_dict =data_df.to_dict(orient='index')

In [23]:
# df = df_counties_original.groupby(['county', 'state'], sort=False)[['cases', 'deaths']].max().sort_values("cases", ascending=False)
# county_state_list = list(map(lambda x: x[0]+", "+x[1], df.index.tolist()))
# meta = {"dates": dt_range_str,"states": county_state_list, "cases": df["cases"].values.tolist(), "deaths": df["deaths"].values.tolist()}
# data_dict["metadata"] = meta

In [24]:
# data_dict

In [25]:
# import json
# with open('./nyt_counties_data.json', 'w') as outfile:
#     json.dump(data_dict, outfile)